In [1]:
import os
from copy import deepcopy

from tqdm import tqdm
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from Environment import Environment, make_one_hot, give_mapping
from Agent import Agent

In [2]:
def give_prev_steps(prev_steps_assist, steps):
    prev_steps_assist = [[0,0,0,0,-1,-1] for i in range(steps-1)]
    return prev_steps_assist

In [3]:
N = 10
batch_size = 5
n_epochs = 4
alpha = 0.0003
memory_len = 6

agent = Agent(gamma=0.99, alpha=0.0003, gae_lambda=0.95, policy_clip=0.2,\
              batch_size=batch_size, N=N, n_epochs=n_epochs, memory_len = memory_len)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice (Tens [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, 2)]          0           input_1[0][0]                    
__________________________________________________________________________________________________
subtract (Subtract)             (None, 2)            0           tf_op_layer_strided_slice[0][0]  
                                                                 tf_op_layer_strided_sl

In [4]:
n_games = 10000
best_score = -50
score_history = []

learn_iters = 0
avg_score = 0
n_steps = 0
max_steps = 40
reached = 0

for i in tqdm(range(n_games)):
    env = Environment()
    env_cell_mapping = give_mapping(env.cells)
    layout = env_cell_mapping[np.newaxis, :, :, np.newaxis]
    done = False
    episode_reward = 0
    start, dest = env.give_start_dest()
    user_state = [start[0], start[1], dest[0], dest[1]]
    prev_steps_assist = []
    prev_steps_assist = give_prev_steps(prev_steps_assist, memory_len)
    step = 0
    episode_reward = 0
    curr_loc = user_state[:2]
    target_loc = user_state[2:]
    
    while not done and step<max_steps:
        step+=1
        n_steps+=1
        user_action, user_prob, asst_action, asst_prob, vals, asst_state, asst_output_one_hot\
        = agent.choose_action(user_state, layout, prev_steps_assist)
        
        
        new_loc, reward, _, done = env.step(user_action, asst_action + 1, target_loc, curr_loc)
        
        if done:
            reached+=1
        
        agent.remember(user_state, asst_state, layout, asst_output_one_hot,\
                       user_action, user_prob, asst_action, asst_prob, vals, reward, done)
         
        user_state = new_loc + target_loc
        prev_steps_assist = asst_state[1:]
        curr_loc = new_loc
        episode_reward+=reward
        
        if n_steps%N==0:
            agent.learn()
            learn_iters+=1
            
    score_history.append(episode_reward)
    avg_score = np.mean(score_history[-20:])
    
    if avg_score>best_score:
        best_score = avg_score
        agent.save_models()
    
    if i%50==0:
        max_steps-=1
        max_steps = max(max_steps, 6)
    
    if i%100==0:
        print(f'Episode: {i}; Score: {episode_reward}; Avg_score: {avg_score}; Timesteps: {n_steps}; Learning Steps: {learn_iters}; Reached: {reached}')
        reached = 0

  0%|                                                                             | 1/10000 [00:04<11:47:28,  4.25s/it]

Saving Models
Episode: 0; Score: -40; Avg_score: -40.0; Timesteps: 40; Learning Steps: 4; Reached: 0


  0%|                                                                             | 2/10000 [00:06<10:02:09,  3.61s/it]

Saving Models


  0%|                                                                              | 4/10000 [00:09<7:05:05,  2.55s/it]

...

 98%|█████████████████████████████████████████████████████████████████████████▌ | 9802/10000 [1:02:45<00:51,  3.87it/s]

Episode: 9800; Score: 8; Avg_score: 7.0; Timesteps: 57013; Learning Steps: 5701; Reached: 96


 99%|██████████████████████████████████████████████████████████████████████████▎| 9901/10000 [1:03:12<00:24,  3.99it/s]

Episode: 9900; Score: 6; Avg_score: 5.8; Timesteps: 57426; Learning Steps: 5742; Reached: 97


100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [1:03:42<00:00,  2.62it/s]


In [6]:
for i in tqdm(range(2)):
    print('----------------------------------------------------------------------------------')
    env = Environment()
    env_cell_mapping = give_mapping(env.cells)
    layout = env_cell_mapping[np.newaxis, :, :, np.newaxis]
    done = False
    episode_reward = 0
    start, dest = env.give_start_dest()
    user_state = [start[0], start[1], dest[0], dest[1]]
    prev_steps_assist = []
    prev_steps_assist = give_prev_steps(prev_steps_assist, memory_len)
    step = 0
    episode_reward = 0
    curr_loc = user_state[:2]
    target_loc = user_state[2:]
    
    while not done and step<max_steps:
        print(user_state)
        step+=1
        n_steps+=1
        user_action, user_prob, asst_action, asst_prob, vals, asst_state, asst_output_one_hot\
        = agent.choose_action(user_state, layout, prev_steps_assist)
        
        print(asst_prob)
        
        new_loc, reward, _, done = env.step(user_action, asst_action + 1, target_loc, curr_loc)
        
        if done:
            reached+=1
        
        agent.remember(user_state, asst_state, layout, asst_output_one_hot,\
                       user_action, user_prob, asst_action, asst_prob, vals, reward, done)
         
        user_state = new_loc + target_loc
        prev_steps_assist = asst_state[1:]
        curr_loc = new_loc
        episode_reward+=reward

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

----------------------------------------------------------------------------------
[0.2, 0.1, 0.0, 0.7]


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  3.88it/s]

[0.2, 0.5, 0.0, 0.7]
[0.0, 0.5, 0.0, 0.7]
[0.0, 0.9, 0.0, 0.7]
[0.0, 0.8, 0.0, 0.7]
----------------------------------------------------------------------------------
[0.4, 0.2, 0.7, 0.3]
[0.8, 0.2, 0.7, 0.3]
[0.4, 0.2, 0.7, 0.3]
[0.8, 0.2, 0.7, 0.3]
[0.7, 0.2, 0.7, 0.3]
